# KALMAN FILTER

### Measure update step

In [2]:
# Write a program to update your mean and variance
# when given the mean and variance of your belief
# and the mean and variance of your measurement.
# This program will update the parameters of your
# belief function.

def update(mean1, var1, mean2, var2):
    new_mean =  (var2 * mean1 + var1 * mean2) / (var1 + var2)
    new_var = 1 / (1/var1 + 1/var2)
    return [new_mean, new_var]

print (update(10.,8.,13., 2.))

[12.4, 1.6]


### Motion predict

In [4]:
# Write a program that will predict your new mean
# and variance given the mean and variance of your 
# prior belief and the mean and variance of your 
# motion. 

def predict(mean1, var1, mean2, var2):
    new_mean = mean1 + mean2
    new_var = var1 + var2
    return [new_mean, new_var]

print (predict(10., 4., 12., 4.))

[22.0, 8.0]


### Full kalman filter



In [6]:
# Write a program that will iteratively update and
# predict based on the location measurements 
# and inferred motions shown below. 

def update(mean1, var1, mean2, var2):
    new_mean = float(var2 * mean1 + var1 * mean2) / (var1 + var2)
    new_var = 1./(1./var1 + 1./var2)
    return [new_mean, new_var]

def predict(mean1, var1, mean2, var2):
    new_mean = mean1 + mean2
    new_var = var1 + var2
    return [new_mean, new_var]

measurements = [5., 6., 7., 9., 10.]
motion = [1., 1., 2., 1., 1.]
measurement_sig = 4.
motion_sig = 2.
mu = 0.
sig = 10000.

#Please print out ONLY the final values of the mean
#and the variance in a list [mu, sig]. 

# Insert code here

for i in range(len(measurements)):

    mu, sig = update(mu, sig, measurements[i] , measurement_sig)
    mu, sig = predict(mu, sig, motion[i], motion_sig)
        
    #print "update" + str([mu, sig])
    #print "predict" + str([mu_s, sig_s])

print ([mu, sig])


[10.999906177177365, 4.005861580844194]


### Multi dimensional Kalman filtere

In [13]:
# Write a function 'kalman_filter' that implements a multi-
# dimensional Kalman Filter for the example given

from math import *

class matrix:
    
    # implements basic operations of a matrix class
    
    def __init__(self, value):
        self.value = value
        self.dimx = len(value)
        self.dimy = len(value[0])
        if value == [[]]:
            self.dimx = 0
    
    def zero(self, dimx, dimy):
        # check if valid dimensions
        if dimx < 1 or dimy < 1:
            raise (ValueError, "Invalid size of matrix")
        else:
            self.dimx = dimx
            self.dimy = dimy
            self.value = [[0 for row in range(dimy)] for col in range(dimx)]
    
    def identity(self, dim):
        # check if valid dimension
        if dim < 1:
            raise (ValueError, "Invalid size of matrix")
        else:
            self.dimx = dim
            self.dimy = dim
            self.value = [[0 for row in range(dim)] for col in range(dim)]
            for i in range(dim):
                self.value[i][i] = 1
    
    def show(self):
        for i in range(self.dimx):
            print (self.value[i])
        print (' ')
    
    def __add__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise (ValueError, "Matrices must be of equal dimensions to add")
        else:
            # add if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] + other.value[i][j]
            return res
    
    def __sub__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise (ValueError, "Matrices must be of equal dimensions to subtract")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] - other.value[i][j]
            return res
    
    def __mul__(self, other):
        # check if correct dimensions
        if self.dimy != other.dimx:
            raise (ValueError, "Matrices must be m*n and n*p to multiply")
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, other.dimy)
            for i in range(self.dimx):
                for j in range(other.dimy):
                    for k in range(self.dimy):
                        res.value[i][j] += self.value[i][k] * other.value[k][j]
            return res
    
    def transpose(self):
        # compute transpose
        res = matrix([[]])
        res.zero(self.dimy, self.dimx)
        for i in range(self.dimx):
            for j in range(self.dimy):
                res.value[j][i] = self.value[i][j]
        return res
    
    # Thanks to Ernesto P. Adorio for use of Cholesky and CholeskyInverse functions
    
    def Cholesky(self, ztol=1.0e-5):
        # Computes the upper triangular Cholesky factorization of
        # a positive definite matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        for i in range(self.dimx):
            S = sum([(res.value[k][i])**2 for k in range(i)])
            d = self.value[i][i] - S
            if abs(d) < ztol:
                res.value[i][i] = 0.0
            else:
                if d < 0.0:
                    raise (ValueError, "Matrix not positive-definite")
                res.value[i][i] = sqrt(d)
            for j in range(i+1, self.dimx):
                S = sum([res.value[k][i] * res.value[k][j] for k in range(self.dimx)])
                if abs(S) < ztol:
                    S = 0.0
                res.value[i][j] = (self.value[i][j] - S)/res.value[i][i]
        return res
    
    def CholeskyInverse(self):
        # Computes inverse of matrix given its Cholesky upper Triangular
        # decomposition of matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        # Backward step for inverse.
        for j in reversed(range(self.dimx)):
            tjj = self.value[j][j]
            S = sum([self.value[j][k]*res.value[j][k] for k in range(j+1, self.dimx)])
            res.value[j][j] = 1.0/tjj**2 - S/tjj
            for i in reversed(range(j)):
                res.value[j][i] = res.value[i][j] = -sum([self.value[i][k]*res.value[k][j] for k in range(i+1, self.dimx)])/self.value[i][i]
        return res
    
    def inverse(self):
        aux = self.Cholesky()
        res = aux.CholeskyInverse()
        return res
    
    def __repr__(self):
        return repr(self.value)


########################################

# Implement the filter function below

def kalman_filter(x, P):
    for n in range(len(measurements)):
        
        # measurement update
        Z = matrix([[measurements[n]]])
        Y = Z - (H * x)
        S = H * P * H.transpose() + R
        K = P * H.transpose() * S.inverse()
        x = x + (K * Y)
        P = (I - (K * H)) * P
        

        # prediction
        x = F*x + u
        P = F * P * F.transpose()
        
    return x,P

############################################
### use the code below to test your filter!
############################################

measurements = [1, 2, 3]

x = matrix([[0.], [0.]]) # initial state (location and velocity)
P = matrix([[1000., 0.], [0., 1000.]]) # initial uncertainty
u = matrix([[0.], [0.]]) # external motion
F = matrix([[1., 1.], [0, 1.]]) # next state function
H = matrix([[1., 0.]]) # measurement function
R = matrix([[1.]]) # measurement uncertainty
I = matrix([[1., 0.], [0., 1.]]) # identity matrix

print( kalman_filter(x, P))
# output should be:
# x: [[3.9996664447958645], [0.9999998335552873]]
# P: [[2.3318904241194827, 0.9991676099921091], [0.9991676099921067, 0.49950058263974184]]


([[3.9996664447958645], [0.9999998335552873]], [[2.3318904241194827, 0.9991676099921091], [0.9991676099921067, 0.49950058263974184]])


# Kalman filter in C++

    // Write a function 'filter()' that implements a multi-
    // dimensional Kalman Filter for the example given
    //============================================================================
    #include <iostream>
    #include "Dense"
    #include <vector>

    using namespace std;
    using namespace Eigen;

    //Kalman Filter variables
    VectorXd x;	// object state
    MatrixXd P;	// object covariance matrix
    VectorXd u;	// external motion
    MatrixXd F; // state transition matrix
    MatrixXd H;	// measurement matrix
    MatrixXd R;	// measurement covariance matrix
    MatrixXd I; // Identity matrix
    MatrixXd Q;	// process covariance matrix

    vector<VectorXd> measurements;
    void filter(VectorXd &x, MatrixXd &P);


    int main() {
        /**
         * Code used as example to work with Eigen matrices
         */
    //	//you can create a  vertical vector of two elements with a command like this
    //	VectorXd my_vector(2);
    //	//you can use the so called comma initializer to set all the coefficients to some values
    //	my_vector << 10, 20;
    //
    //
    //	//and you can use the cout command to print out the vector
    //	cout << my_vector << endl;
    //
    //
    //	//the matrices can be created in the same way.
    //	//For example, This is an initialization of a 2 by 2 matrix
    //	//with the values 1, 2, 3, and 4
    //	MatrixXd my_matrix(2,2);
    //	my_matrix << 1, 2,
    //			3, 4;
    //	cout << my_matrix << endl;
    //
    //
    //	//you can use the same comma initializer or you can set each matrix value explicitly
    //	// For example that's how we can change the matrix elements in the second row
    //	my_matrix(1,0) = 11;    //second row, first column
    //	my_matrix(1,1) = 12;    //second row, second column
    //	cout << my_matrix << endl;
    //
    //
    //	//Also, you can compute the transpose of a matrix with the following command
    //	MatrixXd my_matrix_t = my_matrix.transpose();
    //	cout << my_matrix_t << endl;
    //
    //
    //	//And here is how you can get the matrix inverse
    //	MatrixXd my_matrix_i = my_matrix.inverse();
    //	cout << my_matrix_i << endl;
    //
    //
    //	//For multiplying the matrix m with the vector b you can write this in one line as let’s say matrix c equals m times v.
    //	//
    //	MatrixXd another_matrix;
    //	another_matrix = my_matrix*my_vector;
    //	cout << another_matrix << endl;


        //design the KF with 1D motion
        x = VectorXd(2);
        x << 0, 0;

        P = MatrixXd(2, 2);
        P << 1000, 0, 0, 1000;

        u = VectorXd(2);
        u << 0, 0;

        F = MatrixXd(2, 2);
        F << 1, 1, 0, 1;

        H = MatrixXd(1, 2);
        H << 1, 0;

        R = MatrixXd(1, 1);
        R << 1;

        I = MatrixXd::Identity(2, 2);

        Q = MatrixXd(2, 2);
        Q << 0, 0, 0, 0;

        //create a list of measurements
        VectorXd single_meas(1);
        single_meas << 1;
        measurements.push_back(single_meas);
        single_meas << 2;
        measurements.push_back(single_meas);
        single_meas << 3;
        measurements.push_back(single_meas);

        //call Kalman filter algorithm
        filter(x, P);

        return 0;

    }

    void filter(VectorXd &x, MatrixXd &P) {

        for (unsigned int n = 0; n < measurements.size(); ++n) {

            VectorXd z = measurements[n];
            //YOUR CODE HERE
            // KF Measurement update step
            MatrixXd Y = z - H * x;
            MatrixXd S = H * P * H.transpose() + R;
            MatrixXd K = P * H.transpose() * S.inverse();
            
            // new state
            x = x + (K * Y);
            P = (I - K * H) * P;

            // KF Prediction step
            x = F*x + u;
            P = F*P*F.transpose() + Q;

            std::cout << "x=" << std::endl <<  x << std::endl;
            std::cout << "P=" << std::endl <<  P << std::endl;


        }
    }

# Laser measurment

    #include "Dense"
    #include <iostream>
    #include "tracking.h"

    using namespace std;
    using Eigen::MatrixXd;
    using Eigen::VectorXd;
    using std::vector;

    Tracking::Tracking() {
        is_initialized_ = false;
        previous_timestamp_ = 0;

        //create a 4D state vector, we don't know yet the values of the x state
        kf_.x_ = VectorXd(4);

        //state covariance matrix P
        kf_.P_ = MatrixXd(4, 4);
        kf_.P_ << 1, 0, 0, 0,
                  0, 1, 0, 0,
                  0, 0, 1000, 0,
                  0, 0, 0, 1000;


        //measurement covariance
        kf_.R_ = MatrixXd(2, 2);
        kf_.R_ << 0.0225, 0,
                  0, 0.0225;

        //measurement matrix
        kf_.H_ = MatrixXd(2, 4);
        kf_.H_ << 1, 0, 0, 0,
                  0, 1, 0, 0;

        //the initial transition matrix F_
        kf_.F_ = MatrixXd(4, 4);
        kf_.F_ << 1, 0, 1, 0,
                  0, 1, 0, 1,
                  0, 0, 1, 0,
                  0, 0, 0, 1;

        //set the acceleration noise components
        noise_ax = 5;
        noise_ay = 5;

    }

    Tracking::~Tracking() {

    }

    // Process a single measurement
    void Tracking::ProcessMeasurement(const MeasurementPackage &measurement_pack) {
        if (!is_initialized_) {
            //cout << "Kalman Filter Initialization " << endl;

            //set the state with the initial location and zero velocity
            kf_.x_ << measurement_pack.raw_measurements_[0], measurement_pack.raw_measurements_[1], 0, 0;

            previous_timestamp_ = measurement_pack.timestamp_;
            is_initialized_ = true;
            return;
        }

        //compute the time elapsed between the current and previous measurements
        float dt = (measurement_pack.timestamp_ - previous_timestamp_) / 1000000.0;	//dt - expressed in seconds
        previous_timestamp_ = measurement_pack.timestamp_;

        float dt_2 = dt * dt;
        float dt_3 = dt_2 * dt;
        float dt_4 = dt_3 * dt;

        //Modify the F matrix so that the time is integrated
        kf_.F_(0, 2) = dt;
        kf_.F_(1, 3) = dt;

        //set the process covariance matrix Q
        kf_.Q_ = MatrixXd(4, 4);
        kf_.Q_ <<  dt_4/4*noise_ax, 0, dt_3/2*noise_ax, 0,
                   0, dt_4/4*noise_ay, 0, dt_3/2*noise_ay,
                   dt_3/2*noise_ax, 0, dt_2*noise_ax, 0,
                   0, dt_3/2*noise_ay, 0, dt_2*noise_ay;

        //predict
        kf_.Predict();

        //measurement update
        kf_.Update(measurement_pack.raw_measurements_);

        std::cout << "x_= " << kf_.x_ << std::endl;
        std::cout << "P_= " << kf_.P_ << std::endl;

    }


# jacobian calculation matrix

    #include <iostream>
    #include "Dense"
    #include <vector>

    using namespace std;
    using Eigen::MatrixXd;
    using Eigen::VectorXd;

    MatrixXd CalculateJacobian(const VectorXd& x_state);

    int main() {

        /*
         * Compute the Jacobian Matrix
         */

        //predicted state  example
        //px = 1, py = 2, vx = 0.2, vy = 0.4
        VectorXd x_predicted(4);
        x_predicted << 1, 2, 0.2, 0.4;

        MatrixXd Hj = CalculateJacobian(x_predicted);

        cout << "Hj:" << endl << Hj << endl;

        return 0;
    }

    MatrixXd CalculateJacobian(const VectorXd& x_state) {

        MatrixXd Hj(3,4);
        //recover state parameters
        float px = x_state(0);
        float py = x_state(1);
        float vx = x_state(2);
        float vy = x_state(3);

        //pre-compute a set of terms to avoid repeated calculation
        float c1 = px*px+py*py;
        float c2 = sqrt(c1);
        float c3 = (c1*c2);

        //check division by zero
        if(fabs(c1) < 0.0001){
            cout << "CalculateJacobian () - Error - Division by Zero" << endl;
            return Hj;
        }

        //compute the Jacobian matrix
        Hj << (px/c2), (py/c2), 0, 0,
              -(py/c1), (px/c1), 0, 0,
              py*(vx*py - vy*px)/c3, px*(px*vy - py*vx)/c3, px/c2, py/c2;

        return Hj;
    }

# Evaluate the performance 

    #include <iostream>
    #include "Dense"
    #include <vector>

    using namespace std;
    using Eigen::MatrixXd;
    using Eigen::VectorXd;
    using std::vector;

    VectorXd CalculateRMSE(const vector<VectorXd> &estimations,
            const vector<VectorXd> &ground_truth);

    int main() {
        /*
         * Compute RMSE
         */
        vector<VectorXd> estimations;
        vector<VectorXd> ground_truth;

        //the input list of estimations
        VectorXd e(4);
        e << 1, 1, 0.2, 0.1;
        estimations.push_back(e);
        e << 2, 2, 0.3, 0.2;
        estimations.push_back(e);
        e << 3, 3, 0.4, 0.3;
        estimations.push_back(e);

        //the corresponding list of ground truth values
        VectorXd g(4);
        g << 1.1, 1.1, 0.3, 0.2;
        ground_truth.push_back(g);
        g << 2.1, 2.1, 0.4, 0.3;
        ground_truth.push_back(g);
        g << 3.1, 3.1, 0.5, 0.4;
        ground_truth.push_back(g);

        //call the CalculateRMSE and print out the result
        cout << CalculateRMSE(estimations, ground_truth) << endl;


        return 0;
    }

    VectorXd CalculateRMSE(const vector<VectorXd> &estimations,
            const vector<VectorXd> &ground_truth){

        VectorXd rmse(4);
        rmse << 0,0,0,0;

        // check the validity of the following inputs:
        //  * the estimation vector size should not be zero
        //  * the estimation vector size should equal ground truth vector size
        if(estimations.size() != ground_truth.size()
                || estimations.size() == 0){
            cout << "Invalid estimation or ground_truth data" << endl;
            return rmse;
        }

        //accumulate squared residuals
        for(unsigned int i=0; i < estimations.size(); ++i){

            VectorXd residual = estimations[i] - ground_truth[i];

            //coefficient-wise multiplication
            residual = residual.array()*residual.array();
            rmse += residual;
        }

        //calculate the mean
        rmse = rmse/estimations.size();

        //calculate the squared root
        rmse = rmse.array().sqrt();

        //return the result
        return rmse;
    }
